In [1]:
#############################################################################################
#Как работает данная модель сверточной нейронной сети: 
#    1. По адресу: https://drive.google.com/drive/folders/1l5u9mPdch9VIo3cR6lxFhzmdPKpkbq6j?usp=drive_link - расположены .wav-файлы со звуком плохой фистулы  (для работы модели файлы сохранены локально в папку "../fistulas_sound/bad", всего 8 аудио-файлов
#    2. По адресу: https://drive.google.com/drive/folders/1rVwIXQ78cUMnD_4EsflScot9cVU9BEWZ?usp=drive_link - расположены .wav-файлы со звуком хорошей фистулы (для работы модели файлы сохранены локально в папку "../fistulas_sound/good", всего 43 аудио-файла 
#    3. По адресу: https://drive.google.com/drive/folders/1_PIhhHkwD21lkeGVZHWUx3qGvekXOaNO?usp=drive_link - расположены заархивированные .wav-файлы, подлежащие классификации (для работы модели файлы сохранены локально в папку "../fistulas_sound/vita-fs-data", всего 750 аудио-файлов, т.е. файлы с другими расширениями удаляем, оставляем только .wav)
#    4. С помощью библиотеки librosa тренировочные аудио-файлы (п.1-2) из формата .wav перегоняются в формат .png (спектрограммы) и сохраняются в папки "../img_data/good" (43 шт.) и "../img_data/bad" (8 шт.) соответственно.
#    5. Разбиваем исходный, классифицированный ранее, набор данных (п.1-2) на обучающую и тестовую выборки в соотношении 80/20: для этого сохраняем в папки "../data/train/bad" (6 "плохих" спектрограмм из 8) и "../data/train/good" (34 "хороших" спектрограммы из 43) 80% исходного набора данных, а в папки "../data/val/bad" (2 "плохих" из 8) и "../data/val/good" (9 "хороших" из 43) - 20% исходного набора данных.
#    6. Используя класс Sequential() из keras формируем модель свёрточной нейронной сети (convolutional neural network (CNN)).
#    7. Обучаем модель с применением оптимизатора - стохастического градиентного спуска (SGD).
#    8. Производим оценку модели.
#    9. Переносим максимальное кол-во файлов, кратное размеру пачки (в данном случае - 5) в формат .png действиями, аналогичными действиям из п.4. Изображения спектрограмм сохраняем в папку "../data/test/unclassified".   
#    10. Выполняем классификацию на основном тестовом наборе данных (750 спектрограмм из папки "../data/test/unclassified").
#    11. Сопоставляем результаты классификации.
#    12. Записываем результат бинарной классификации в файл "prediction_results_CNN_model_spec.csv".
#############################################################################################

In [2]:
#п.1-3 делаем до запуска IDE (модель сформирована в локальном jupyter-notebook.
#Далее в листинге в комментах будет указан номер пункта из описанной выше работы модели.

In [3]:
#импортируем нужные библиотеки
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split

In [4]:
#подключаем tensorflow, keras - в его составе
import tensorflow as tf
from tensorflow import keras

In [5]:
#подключаем элементы keras
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD #градиентный спуск

In [6]:
#п.4 описания - формируем директорию img_data, содержащую все изображения, классифицированные по видам фистул: плохая/хорошая
fistulas_kind = 'good bad'.split()
for f in fistulas_kind:
    pathlib.Path(f'img_data/{f}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'fistulas_sound/{f}/'):
        wav_name = f'fistulas_sound/{f}/{filename}'
        y, sr = librosa.load(wav_name, sr = 22050, mono=True, duration=5)
        #print(y)
        plt.figure(figsize=(12,4))
        librosa.display.waveshow(y, sr=sr)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap='Blues', sides='default', mode='default', scale='dB');
        #plt.axis('off');
        plt.savefig(f'img_data/{f}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

C:\Users\Chernyavskaya_OS\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\axes\_axes.py:7720: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)
C:\Users\Chernyavskaya_OS\AppData\Local\Temp\ipykernel_14084\273696983.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12,4))


<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

...

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

In [7]:
import splitfolders
#п.5 описания - разбиваем на обучающую и тестовую выборки: 80/20
splitfolders.ratio('./img_data/', output="./data", seed=1337, ratio=(.8, .2)) # значения по умолчанию

Copying files: 51 files [00:00, 63.89 files/s]


In [8]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255, #изменение масштаба всех значений пикселей с 0 до 255, после этого шага они будут находится в диапазоне (0..1)
        shear_range=0.2, #применение случайных преобразований
        zoom_range=0.2, #увеличение масштаба
        horizontal_flip=True) #горизонтальный поворот
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
training_set = train_datagen.flow_from_directory(
        './data/train',
        target_size=(64, 64),
        batch_size=5,
        class_mode='categorical',
        shuffle = False)
test_set = test_datagen.flow_from_directory(
        './data/val',
        target_size=(64, 64),
        batch_size=5,
        class_mode='categorical',
        shuffle = False ) 

Found 40 images belonging to 2 classes.
Found 11 images belonging to 2 classes.


In [10]:
#п.6 описания - формируем модель свёрточной нейронной сети:
model_CNN = Sequential()
input_shape=(64, 64, 3)

#первый скрытый слой
model_CNN.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model_CNN.add(AveragePooling2D((2, 2), strides=(2,2)))
model_CNN.add(Activation('relu'))

#второй скрытый слой
model_CNN.add(Conv2D(64, (3, 3), padding="same"))
model_CNN.add(AveragePooling2D((2, 2), strides=(2,2)))
model_CNN.add(Activation('relu'))

#третий скрытый слой
model_CNN.add(Conv2D(64, (3, 3), padding="same"))
model_CNN.add(AveragePooling2D((2, 2), strides=(2,2)))
model_CNN.add(Activation('relu'))

#слой выравнивания
model_CNN.add(Flatten())
model_CNN.add(Dropout(rate=0.5))

#полносвязный слой
model_CNN.add(Dense(64))
model_CNN.add(Activation('relu'))
model_CNN.add(Dropout(rate=0.5))

#выходной слой
model_CNN.add(Dense(2))
model_CNN.add(Activation('softmax'))
model_CNN.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 31, 32)        896       
                                                                 
 average_pooling2d (Average  (None, 15, 15, 32)        0         
 Pooling2D)                                                      
                                                                 
 activation (Activation)     (None, 15, 15, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 64)        18496     
                                                                 
 average_pooling2d_1 (Avera  (None, 7, 7, 64)          0         
 gePooling2D)                                                    
                                                                 
 activation_1 (Activation)   (None, 7, 7, 64)          

In [11]:
#п.7 описания - обучаем модель - команда compile
epochs = 100
batch_size = 5
learning_rate = 0.01
decay_rate = learning_rate/epochs
momentum = 0.9
sgd = SGD(learning_rate=learning_rate, momentum=momentum, weight_decay=decay_rate, nesterov=False) #sgd - это стох.град.спуск
model_CNN.compile(optimizer="sgd", loss=keras.losses.Hinge(), metrics=['accuracy'])

In [12]:
model_CNN.fit_generator(
        training_set,
        steps_per_epoch=1,
        epochs=1,
        validation_data=test_set,
        validation_steps=1)

C:\Users\Chernyavskaya_OS\AppData\Local\Temp\ipykernel_14084\2455810477.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_CNN.fit_generator(




1/1 [==============================] - 7s 7s/step - loss: 1.0051 - accuracy: 0.2000 - val_loss: 0.9989 - val_accuracy: 0.6000


In [13]:
#п.8 описания - оценка модели
model_CNN.evaluate_generator(generator=test_set, steps=1)

C:\Users\Chernyavskaya_OS\AppData\Local\Temp\ipykernel_14084\3211313860.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_CNN.evaluate_generator(generator=test_set, steps=1)


[0.9988595247268677, 0.6000000238418579]

In [14]:
#п.9 описания - переносим основные данные wav->png для классификации с пом.построенной модели (переношу 750 файлов, размер пачки = 5 изображений),
#для вывода рез-тов классиф-ии в файл нужно, чтобы совпадало кол-во сэмплов и рез-тов (это 2 столбца в результирующем .csv)
li_cnt = 0 #счётчик перенесенных файлов
pathlib.Path(f'data/test/unclassified').mkdir(parents=True, exist_ok=True)
for filename in os.listdir(f'fistulas_sound/vita-fs-data/'):
    if li_cnt <= 750:
        try:
            wav_name = f'fistulas_sound/vita-fs-data/{filename}'
            y, sr = librosa.load(wav_name, sr=22050, mono=True, duration=5)
            #plt.axis('off');
            plt.figure(figsize=(12,4))
            librosa.display.waveshow(y, sr=sr)
            plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap='Blues', sides='default', mode='default', scale='dB');
            plt.savefig(f'data/test/unclassified/{filename[:-3].replace(".", "")}.png')
            plt.clf()
            li_cnt = li_cnt + 1
        except:
            print(filename)
    else:
        break

C:\Users\Chernyavskaya_OS\AppData\Local\Temp\ipykernel_14084\2521482018.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12,4))


<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

...

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>

In [16]:
main_test_set = test_datagen.flow_from_directory(
        './data/test',
        target_size=(64, 64),
        batch_size=5,
        class_mode='categorical',
        shuffle = False )

Found 750 images belonging to 1 classes.


In [17]:
#п.11 описания - выполняем прогноз на основном тестовом наборе данных (750 спектрограмм).
main_test_set.reset()
pred = model_CNN.predict_generator(main_test_set, steps=150, verbose=1)

C:\Users\Chernyavskaya_OS\AppData\Local\Temp\ipykernel_14084\1333899310.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = model_CNN.predict_generator(main_test_set, steps=150, verbose=1)


150/150 [==============================] - 61s 342ms/step


In [18]:
pred

array([[0.495183  , 0.50481695],
       [0.49634838, 0.50365156],
       [0.49597692, 0.5040231 ],
       ...,
       [0.49380428, 0.5061957 ],
       [0.50190204, 0.4980979 ],
       [0.49358332, 0.50641674]], dtype=float32)

In [19]:
#п.12 описания - сопоставляем рез-ты классификации
ar = np.array
for l in pred:
    if l[0] > l[1]:
        ar = np.append(ar, 1)
    else:
        ar = np.append(ar, 0)
ar = np.delete(ar, 0)
predicted_class_indices=ar

In [20]:
labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:750]
filenames=main_test_set.filenames

In [21]:
#проверка - кол-во классифицируемых записей д.б. = кол-ву выполненных прогнозов.
print(len(filenames), len(predictions))

750 750


In [22]:
#п.13 описания - записываем рез-т в файл "prediction_results_CNN_model_spec.csv"
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("prediction_results_CNN_model_spec.csv",index=False)